In [3]:
import requests
from urllib.parse import quote

def get_lyrics(artist: str, title: str, timeout: int = 15) -> str | None:
    # Encode para espacios y caracteres especiales
    url = f"https://api.lyrics.ovh/v1/{quote(artist)}/{quote(title)}"
    r = requests.get(url, timeout=timeout)

    if r.status_code == 200:
        data = r.json()
        return data.get("lyrics")
    if r.status_code == 404:
        return None

    # Otros códigos (500, 429, etc.)
    r.raise_for_status()

lyrics = get_lyrics("Tony Orlando","Say, Has Anybody Seen My Sweet Gypsy Rose")
print(lyrics)


None


In [25]:
import requests
from bs4 import BeautifulSoup

years = list(range(1973, 2023))
base_url = "https://en.wikipedia.org/wiki/Billboard_Year-End_Hot_100_singles_of_"

songs = {}
singers = {}

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120 Safari/537.36"
}

for year in years:
    names = []
    singer_names = []

    url = f"{base_url}{year}"
    r = requests.get(url, headers=HEADERS, timeout=20)

    # Si Wikipedia no responde OK, saltá ese año (y lo logueás)
    if r.status_code != 200:
        print(f"[WARN] {year} status={r.status_code} url={url}")
        songs[year] = names
        singers[year] = singer_names
        continue

    soup = BeautifulSoup(r.text, "html.parser")

    tables = soup.find_all("table", class_="wikitable")
    if not tables:
        # Esto evita el IndexError y te avisa qué año falló
        print(f"[WARN] {year} no wikitable found. Possible block/HTML change.")
        songs[year] = names
        singers[year] = singer_names
        continue

    # Opción: elegir la tabla correcta. A veces hay varias wikitables.
    # Normalmente la tabla del ranking es la que tiene columnas "Rank" y "Title"/"Artist".
    target = None
    for t in tables:
        header = t.get_text(" ", strip=True).lower()
        if "rank" in header and ("artist" in header or "artists" in header):
            target = t
            break
    if target is None:
        target = tables[0]  # fallback

    for row in target.find_all("tr"):
        links = row.find_all("a")
        if not links:
            continue

        # Tu lógica original (con un poquito más de cuidado)
        if len(links) > 2:
            song = links[0].get_text(strip=True)
            singer = ",".join([lk.get_text(strip=True) for lk in links[1:]])
        elif len(links) == 2:
            song = links[0].get_text(strip=True)
            singer = links[1].get_text(strip=True)
        else:
            song = links[0].get_text(strip=True)
            singer = None

        if song:  # evita basura
            names.append(song)
            singer_names.append(singer)

    songs[year] = names
    singers[year] = singer_names


In [21]:
 singers


{1973: ['Tony Orlando and Dawn',
  'Jim Croce',
  'Roberta Flack',
  'Marvin Gaye',
  'Paul McCartney & Wings',
  'Kris Kristofferson',
  'Elton John',
  'Billy Preston',
  'Carly Simon',
  'Diana Ross',
  'Vicki Lawrence',
  'Clint Holmes',
  'Stories',
  'Helen Reddy',
  'Billy Paul',
  'The Edgar Winter Group',
  'Dobie Gray',
  'Sweet',
  'Stevie Wonder',
  'Cher',
  'The Isley Brothers',
  'Sylvia Robinson',
  'Grand Funk Railroad',
  'Dr. John',
  'Skylark',
  'Stevie Wonder',
  'Paul Simon',
  'Maureen McGovern',
  'John Denver',
  'Stealers Wheel',
  'Three Dog Night',
  "The O'Jays",
  'Barry White',
  'Tony Orlando and Dawn',
  'Eddie Kendricks',
  'Anne Murray',
  'King Harvest',
  "Bobby 'Boris' Pickett",
  'Bloodstone',
  'Seals and Crofts',
  'The Doobie Brothers',
  'George Harrison',
  'Sly & the Family Stone',
  'Jermaine Jackson',
  'Gladys Knight & the Pips',
  'New York City',
  'The Spinners',
  'Elton John',
  'Gladys Knight & the Pips',
  'Deep Purple',
  'Dr. Ho